# Классификация текстов

В этом ноутбуке мы разберем задачу классификации текстов на примере соревнования по выявлению токсичных твиттов: https://www.kaggle.com/competitions/toxic-comments-classification-2

Наша задача - построить классификатор, который по тексту твитта определяет, токсичный он или нет.

__План:__
1. Разбираем/освежаем в памяти простые бейзлайны: мешок слов, TF-IDF. 
2. Обучаем классификатор на основе w2v эмбеддингов
3. Знакомимся с моделью fastText

In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

Обучающая выборка и тестовые данные для предикта - их можно скачать с Kaggle.

In [10]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [11]:
train.sample(3)

,comment,toxic
10547,Я почему-то прочитал из Морровинда)\n,0.0
3466,"Такая же фигня , утром в 08:00 с зарядки снима...",0.0
945,- А где Вы живёте? - За чертой бедности! ost в...,1.0


In [12]:
test.sample()

,comment_id,comment
2219,2219,"Я тоже немного удивлён, её доля в наследстве 5..."


In [13]:
# y_test = pd.read_csv('test_labels.csv')

В моем распоряжении также есть файл с тестовыми метками классов - если Вы запускаете ноутбук и хотите обучить модель, разбейте train выборку на train и test, раскомментировав строчки кода в ячейке ниже:

In [14]:
train, test = train_test_split(train, test_size=0.3, random_state=1)
y_test = test['toxic']
test.drop(columns=['toxic'], inplace=True)

In [15]:
y_test.sample()

6490    0.0
Name: toxic, dtype: float64

### Мы начнем с простых бейзлайнов

Это всегда хорошая практика - сперва попробовать что-то предельно простое (: В нашем случае это будет логистическая регрессия + мешок слов (Bag of Words, BoW).

In [16]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
vec = CountVectorizer(ngram_range=(1, 1), token_pattern='\w{3,}') # строим BoW для слов

In [18]:
bow = vec.fit_transform(train['comment'])

In [19]:
bow

<7566x43792 sparse matrix of type '<class 'numpy.int64'>'
	with 145425 stored elements in Compressed Sparse Row format>

In [20]:
print(train.comment[10804])

А у мамы в группе до самого выпуска из сада такие просяки нет-нет, да случаются, с разными детьми. Только для одних детей это разовая акция, у других - время от времени, а у третьих - частенько. И родители прекрасно в курсе особенностей каждого конкретного ребенка.



In [21]:
list(vec.vocabulary_.items())[:10]

[('шовинистические', 42944),
 ('пидораны', 25355),
 ('долбоеб', 9303),
 ('чтоле', 42557),
 ('под', 26141),
 ('кроватью', 15811),
 ('куколдов', 16027),
 ('ищи', 13557),
 ('это', 43516),
 ('долгая', 9321)]

In [22]:
sorted(list(vec.vocabulary_.items()), key=lambda x: x[0])[:10]

[('000', 0),
 ('0036', 1),
 ('005', 2),
 ('00х', 3),
 ('013', 4),
 ('030050', 5),
 ('0849', 6),
 ('0х200в', 7),
 ('100', 8),
 ('1000', 9)]

In [23]:
list(vec.vocabulary_.keys())[:10]

['шовинистические',
 'пидораны',
 'долбоеб',
 'чтоле',
 'под',
 'кроватью',
 'куколдов',
 'ищи',
 'это',
 'долгая']

In [24]:
len(vec.vocabulary_.items())

43792

In [25]:
y_train = train['toxic'].astype(int).values
y_train

array([1, 1, 0, ..., 1, 0, 0])

In [26]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)

LogisticRegression(max_iter=500, random_state=42)

In [27]:
len(clf.coef_[0])

43792

In [28]:
bow_test = vec.transform(test['comment'])
bow_test

<3243x43792 sparse matrix of type '<class 'numpy.int64'>'
	with 51429 stored elements in Compressed Sparse Row format>

In [29]:
pred = clf.predict(bow_test)
pred[:10]

array([0, 1, 1, 1, 1, 0, 1, 0, 0, 0])

In [30]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.93      0.84      0.88      2407
           1       0.64      0.83      0.72       836

    accuracy                           0.84      3243
   macro avg       0.79      0.83      0.80      3243
weighted avg       0.86      0.84      0.84      3243



### Попробуем добавить препроцессинг текста

Препроцессинг, как правило, включает удаление небуквенных символов, топ-слов и нормализацию (стемминг - приведение к основе слова - или лемматизацию - приведение слов к начальной форме).

Кроме того, заменим мешок слов на TF-IDF матрицу. В качестве модели оставим логистическую регрессию.

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
# !pip install pymorphy2

In [33]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zver\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Функция для удаления небуквенных символов из текста:

In [34]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

from functools import lru_cache
from tqdm.notebook import tqdm

m = MorphAnalyzer()
regex = re.compile("[а-яa-zёЁ()]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

In [35]:
train.comment[1]

'И именно эти неработающие весы показывают, что работающих нет?..\n'

In [36]:
words_only(train.comment[1])

['и',
 'именно',
 'эти',
 'неработающие',
 'весы',
 'показывают',
 'что',
 'работающих',
 'нет']

Функции для препроцессинга текста: 

1. Удаление небуквенных символов
2. Лемматизация 
3. Удаление коротких (менее 3 символов) и стоп-слов

In [37]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [38]:
%time lemmatize_word('неработающие')

Wall time: 0 ns


'неработающий'

In [39]:
train.comment[1]

'И именно эти неработающие весы показывают, что работающих нет?..\n'

In [40]:
clean_text(train.comment[1])

'именно неработающий весы показывать работать'

Проводим препроцессинг для train и test выборок:

In [41]:
lemmas = list(tqdm(map(clean_text, train['comment']), total=len(train)))
    
train['lemmas'] = lemmas
train.sample(5)

  0%|          | 0/7566 [00:00<?, ?it/s]

,comment,toxic,lemmas
5491,Татаре больше русские чем весь этот хохло-буль...,1.0,татар большой русский весь хохнуть бульбаший у...
1102,Видимо на 5ггц получше ситуация? Возможно. Так...,0.0,видимо хороший ситуация возможно также возможн...
529,"готовься к санкциям Не проецируй, пидораха. Ра...",1.0,готовиться санкция проецировать пидорах разрыв...
8211,"Не, ну он не мог сначала в село перебраться, а...",0.0,мочь сначала село перебраться оформлять успеть...
5537,"Говорят что обиженных в жопу долбят, но это не...",1.0,говорить обиженный жопа долбить точно


In [42]:
lemmas_test = list(tqdm(map(clean_text, test['comment']), total=len(test)))
    
test['lemmas'] = lemmas_test

  0%|          | 0/3243 [00:00<?, ?it/s]

Считаем TF-IDF матрицу и обучаем модель:

In [43]:
vec = TfidfVectorizer(ngram_range=(1, 1)) # строим BoW для слов
tfidf = vec.fit_transform(train['lemmas'])

clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(tfidf, y_train)

pred = clf.predict(vec.transform(test['lemmas']))
accuracy_score(pred, y_test)

0.8294788775824854

In [44]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.97      0.81      0.88      2597
           1       0.54      0.91      0.68       646

    accuracy                           0.83      3243
   macro avg       0.76      0.86      0.78      3243
weighted avg       0.89      0.83      0.84      3243



## Word2Vec

Попробуем использовать эмбеддинги слов - для этого сперва обучим модель Word2Vec c помощью библиотеки gensim.

In [45]:
from gensim.models import word2vec

In [46]:
train.sample()

,comment,toxic,lemmas
8281,"Ну тогда вы должны знать, что при готовке на о...",0.0,должный знать готовка примеру) человек идти ра...


In [ ]:
#help( word2vec.Word2Vec)

In [47]:
tokenized_tweets = [tweet.split() for tweet in train['lemmas'].values]

w2v = word2vec.Word2Vec(
    tokenized_tweets, workers=4, vector_size=200, min_count=10, window=3, sample=1e-3
)

In [48]:
w2v.wv.most_similar(positive=['плюс'], topn=10)

[('работа', 0.9997643232345581),
 ('самый', 0.9997617602348328),
 ('поэтому', 0.999758243560791),
 ('стать', 0.9997527003288269),
 ('делать', 0.9997518062591553),
 ('сразу', 0.9997504353523254),
 ('давать', 0.9997462630271912),
 ('говно', 0.9997437000274658),
 ('второй', 0.9997434020042419),
 ('брать', 0.9997427463531494)]

Теперь у нас есть эмбеддинги для слов. Но как получить эмбеддинги для твитов?

Можно просто усреднить эмбеддинги слов, входящих в твит.

In [50]:
def get_tweet_embedding(lemmas, model=w2v.wv, embedding_size=200):
    
    res = np.zeros(embedding_size)
    cnt = 0
    for word in lemmas.split():
        if word in model:
            res += np.array(model[word])
            cnt += 1
    if cnt:
        res = res / cnt
    return res

In [51]:
get_tweet_embedding('привет тебя')

array([ 0.01031453, -0.00134417,  0.02363954,  0.02723716,  0.05476218,
       -0.02475743, -0.00403957,  0.08161531, -0.02173087,  0.03592744,
       -0.03448499, -0.04116287, -0.02004078,  0.04912569, -0.02209529,
       -0.04795951, -0.02215341, -0.0099673 ,  0.02670055, -0.07311655,
        0.04012465, -0.04025758,  0.01497772,  0.00056953, -0.00118035,
       -0.01436246, -0.00475811, -0.02736854, -0.06704505,  0.02158085,
        0.05515433,  0.01453464,  0.01413451,  0.00273   ,  0.0138288 ,
        0.02710798,  0.04652164, -0.00450038, -0.01570326, -0.07531952,
       -0.03459896, -0.01803993, -0.01393493,  0.0472967 ,  0.09521075,
        0.00203819, -0.00636373, -0.0219558 ,  0.04197318,  0.04680048,
        0.02558938, -0.01729854, -0.0240805 , -0.03913648, -0.00690955,
       -0.01291359,  0.01348682, -0.05639656, -0.0451028 , -0.00132419,
       -0.03010101,  0.0175846 , -0.01651896,  0.01775512, -0.04993922,
        0.02794012,  0.00030038,  0.06533195, -0.04434856,  0.05

Для каждого твита из обучающей и тестовой выборки вычислим такой эмбеддинг:

In [52]:
train['w2v_embedding'] = train['lemmas'].map(get_tweet_embedding)
test['w2v_embedding'] = test['lemmas'].map(get_tweet_embedding)

In [53]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(list(train['w2v_embedding'].values), y_train)

pred = clf.predict(list(test['w2v_embedding'].values))
accuracy_score(pred, y_test)

0.6734505087881592

In [54]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.68      0.80      3127
           1       0.07      0.62      0.12       116

    accuracy                           0.67      3243
   macro avg       0.52      0.65      0.46      3243
weighted avg       0.95      0.67      0.78      3243



## FastText

FastText - это модификация модели word2vec.

FastText использует не только векторы слов, но и векторы n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Руководство](https://fasttext.cc/docs/en/support.html)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

На сайте проекта можно найти предобученные модели для 157 языков (в том числе русского): https://fasttext.cc/docs/en/crawl-vectors.html

Для начала, попробуем взять предобученную модель fastText с сайта проекта и заменить эмбеддинги в модели выше на эмбеддинги fastText.

Бонус: попробуйте взять модель с сайта проекта Rusvetores: https://rusvectores.org/ru/models/

In [ ]:
# !git clone https://github.com/facebookresearch/fastText.git
# !cd fastText
# !sudo pip install .
# !sudo python setup.py install

In [1]:
import fasttext
import fasttext.util

In [ ]:
# help(fasttext.util.download_model)

In [3]:
# fasttext.util.download_model('ru', if_exists='ignore')
ft = fasttext.load_model('cc.ru.300.bin')

In [4]:
ft['привет']

array([ 0.06434693, -0.01527086, -0.06963537, -0.03582602,  0.01471584,
       -0.03503159,  0.02701715,  0.04161827, -0.00033126,  0.00355259,
        0.06979205,  0.06205348,  0.05154078,  0.03831509, -0.02394784,
       -0.03954181, -0.00189653, -0.11174394, -0.0407712 ,  0.09289949,
       -0.07412342, -0.05209147,  0.02017231,  0.04837443,  0.02212641,
        0.00856511, -0.03055364,  0.04733564,  0.04380886,  0.03856769,
        0.03442968,  0.05576854,  0.01513439,  0.14055566,  0.03365337,
       -0.02920472, -0.10305687, -0.09332671,  0.03085899, -0.11067575,
       -0.08992791,  0.05850704, -0.017424  ,  0.00120653, -0.07153153,
        0.10312843, -0.08066262, -0.00642456,  0.04408539, -0.05728461,
       -0.0179531 ,  0.03936698,  0.04778077, -0.04907751, -0.00909553,
        0.05588715, -0.00236535,  0.04878682, -0.01769035,  0.03295048,
        0.00906604,  0.08772802,  0.02970458, -0.04903899, -0.03025401,
       -0.04151824,  0.04931813, -0.02804473,  0.05716789,  0.03

In [5]:
ft.get_nearest_neighbors('лошадь')

[(0.7690380811691284, 'Лошадь'),
 (0.7525508999824524, 'лошадь.'),
 (0.7065301537513733, 'кобылу'),
 (0.7039711475372314, 'лошади'),
 (0.6999924182891846, 'лошадка'),
 (0.6913052797317505, 'кобыла'),
 (0.6855972409248352, 'лошадку'),
 (0.6669025421142578, 'лошаденка'),
 (0.6608322262763977, 'клячу'),
 (0.6564413905143738, 'коня')]

In [55]:
def get_tweet_embedding(lemmas, model=w2v.wv, embedding_size=200):
    
    res = np.zeros(embedding_size)
    cnt = 0
    for word in lemmas.split():
            res += np.array(model[word])
            cnt += 1
    if cnt:
        res = res / cnt
    return res

In [56]:
x = 'привет всем слушателям курса'
get_tweet_embedding(x, model=ft, embedding_size=300)

array([ 2.84749218e-02,  1.14055865e-02, -1.54750008e-02,  6.10717852e-03,
       -5.42343501e-03,  2.83443742e-03,  2.40256451e-03,  1.29073053e-02,
        3.05031866e-02, -1.99234379e-02,  6.13203850e-02,  4.42768331e-02,
        2.71531800e-02, -1.02064133e-02,  9.22483567e-04,  2.50384058e-02,
       -1.25383004e-02, -4.89095808e-02, -3.07890818e-02,  1.01918663e-01,
       -2.85800546e-02, -1.05811988e-01, -1.28629373e-02,  2.95597422e-02,
        2.13206490e-03,  1.26906892e-02, -2.97227059e-02,  2.77029723e-02,
       -1.21254625e-02, -4.76178443e-02, -6.68591424e-03,  3.05985650e-02,
        3.59081652e-02,  1.02970391e-01,  3.62780495e-02, -5.56655712e-02,
       -1.11200343e-01, -1.16946280e-01,  4.69890856e-02, -5.79430675e-02,
       -4.56299540e-03, -2.32621958e-03, -2.30524363e-03,  1.96370891e-02,
       -1.68996924e-02,  4.77626729e-02, -7.71877861e-02,  2.95996453e-02,
        3.40769021e-02, -3.43663241e-02,  5.55797149e-02,  1.05126291e-02,
        9.77615127e-03,  

In [57]:
train['ft_embedding'] = train['lemmas'].apply(lambda x: get_tweet_embedding(x, model=ft, embedding_size=300))
print('train done')

test['ft_embedding'] = test['lemmas'].apply(lambda x: get_tweet_embedding(x, model=ft, embedding_size=300))

train done


In [58]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(list(train['ft_embedding'].values), y_train)

pred = clf.predict(list(test['ft_embedding'].values))
accuracy_score(pred, y_test)

0.8744989207523898

In [59]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.95      0.87      0.91      2361
           1       0.72      0.89      0.79       882

    accuracy                           0.87      3243
   macro avg       0.84      0.88      0.85      3243
weighted avg       0.89      0.87      0.88      3243



### fastText как классификатор

fastText также можно использовать в режиме классификатора:

In [60]:
train.sample()

,comment,toxic,lemmas,w2v_embedding,ft_embedding
1278,"Пока мы в России убеждаем друг друга, что ВИЭ ...",0.0,пока россия убеждать друг друг несерьёзно стра...,"[0.026844523048826625, 0.015457355417311192, 0...","[0.03609138665099939, -0.018015047017898824, 0..."


In [61]:
with open('train_ft.txt', 'w') as f:
    for label, lemmas in list(zip(
        train['toxic'], train['lemmas']
    )):
        f.write(f"__label__{int(label)} {lemmas}\n")
        #print(f"__label__{int(label)} {lemmas}")

with open('test_ft.txt', 'w') as f:
    for label, lemmas in list(zip(
        y_test, test['lemmas']
    )):
        f.write(f"__label__{int(label)} {lemmas}\n")

In [62]:
f = open('train_ft.txt', 'r')
f.readlines(0)[:5]

['__label__1 шовинистический пидоран\n',
 '__label__1 долбоести чтол кровать куколд искать\n',
 '__label__0 долгий история назвать привести эволюция подпись\n',
 '__label__1 продолжить угощать весь говнец пара выпуск хуета александр мочь бабуля живой\n',
 '__label__1 похожий въебливый зануда похожий особенно мочь кроме говно интернет кидаться нарушение правило vovkabright\n']

In [63]:
classifier = fasttext.train_supervised('train_ft.txt')#, 'model')
result = classifier.test('test_ft.txt')
print('P@1:', result[1])#.precision)
print('R@1:', result[2])#.recall)
print('Number of examples:', result[0])#.nexamples)

P@1: 0.8513721862473019
R@1: 0.8513721862473019
Number of examples: 3243


In [64]:
pred = classifier.predict(list(test['lemmas']))[0]
pred = [int(label[0][-1]) for label in pred]

accuracy_score(list(y_test), pred)

0.8513721862473019